# Import Libraries

In [1]:
import numpy as np
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import sys
from pdb import set_trace as pb
from datetime import datetime
import time
from time import mktime          # mktime transforms datetime objects to unix timestamps

# Unix Time Converter Function
To convert format (dd-mm-yyyy) into unix time format

In [2]:
def convert_to_unix(date):
    """
    converts date to unix timestamp    
    parameters: date - in format (dd-mm-yyyy)    
    returns integer unix timestamp
    """
    datum = datetime.strptime(date, '%d-%m-%Y')
    
    return int(mktime(datum.timetuple()))

# Crumb and Cookie Extractor Function
To extract crumb and cookie from yahoo finance webpage

In [3]:
def _get_crumbs_and_cookies(stock):
    """
    get crumb and cookies for historical data csv download from yahoo finance    
    parameters: stock - short-handle identifier of the company     
    returns a tuple of header, crumb and cookie
    """
    
    url = 'https://finance.yahoo.com/quote/{}/history'.format(stock)
    with requests.session():
        header = {'Connection': 'keep-alive',
                   'Expires': '-1',
                   'Upgrade-Insecure-Requests': '1'
                   }        
        website = requests.get(url, headers=header)
        soup = BeautifulSoup(website.text, 'lxml')
        crumb = re.findall('"CrumbStore":{"crumb":"(.+?)"}', str(soup))

        return (header, crumb[0], website.cookies)


# Load CSV Data Function
To download data in csv format from yahoo finance website based on stock code

In [4]:
def load_csv_data(stock, interval='1d', day_begin='01-01-2000', day_end='01-01-2020'):
    """
    queries yahoo finance api to receive historical data in csv file format
    
    parameters: 
        stock - short-handle identifier of the company
        
        interval - 1d, 1wk, 1mo - daily, weekly monthly data
        
        day_begin - starting date for the historical data (format: dd-mm-yyyy)
        
        day_end - final date of the data (format: dd-mm-yyyy)
    
    returns a list of comma seperated value lines
    """
    day_begin_unix = convert_to_unix(day_begin)
    day_end_unix = convert_to_unix(day_end)
    
    header, crumb, cookies = _get_crumbs_and_cookies(stock)    
    with requests.session():
        url = 'https://query1.finance.yahoo.com/v7/finance/download/' \
              '{stock}?period1={day_begin}&period2={day_end}&interval={interval}&events=history&crumb={crumb}' \
              .format(stock=stock, day_begin=day_begin_unix, day_end=day_end_unix, interval=interval, crumb=crumb)
                
        website = requests.get(url, headers=header, cookies=cookies)
       
        return website.text.split('\n')[:-1]
    

# Generate Yahoo Stock Code from Website
Generating the list of yahoo stockcodes from the website

In [5]:
# Source url
source = requests.get('https://sginvestors.io/market/sgx-share-price-performance/s-reits-sector').text
soup = BeautifulSoup(source,'lxml')
lsts = []
# Get co
for td in soup.find_all('td'):
    for a in td.find_all('a'):        
        a_2 = (a.text.split(' ('))
        a_2[1] = a_2[1].split(')')[0].split(':')[1] + '.SI'
        lsts.append(a_2)
df_lsts = pd.DataFrame(lsts)
df_lsts.columns = ['REIT Name','Code']
df_lsts


,REIT Name,Code
0,LIPPO MALLS INDO RETAIL TRUST,D5IU.SI
1,MAPLETREE NORTH ASIA COMM TR,RW0U.SI
2,FRASERS COMMERCIAL TRUST,ND8U.SI
3,CROMWELL EUROPEAN REIT,CNNU.SI
4,FRASERS LOGISTICS & IND TRUST,BUOU.SI
5,PARKWAYLIFE REIT,C2PU.SI
6,SPH REIT,SK6U.SI
7,FRASERS HOSPITALITY TRUST,ACV.SI
8,OUE HOSPITALITY TRUST,SK7.SI
9,KEPPEL-KBS US REIT,CMOU.SI


# Create a stock dataframe dictionary
 Creating a dictionary with stockcode key and downloaded csv list from yahoo finance for stockcodes

In [6]:
codes = df_lsts['Code'].values
dict_stk = {}
# Download csv file from yahoo
for num, code in enumerate(codes, start=1):    
    retry = 0
    while True:
        try:            
            new_list = [line.split(',') for line in load_csv_data(code)] 
            df = pd.DataFrame(new_list[1:], columns=new_list[0])
            df = df.set_index('Date')
            dict_stk[code] = df
            print('{}. {} loaded.'.format(num, code))
            break
        except:
            retry = retry + 1;            
            time.sleep(1)            
            if retry > 10:
                print('Error in converting dataframe in {}'.format(lst[1]))
                break

1. D5IU.SI loaded.
2. RW0U.SI loaded.
3. ND8U.SI loaded.
4. CNNU.SI loaded.
5. BUOU.SI loaded.
6. C2PU.SI loaded.
7. SK6U.SI loaded.
8. ACV.SI loaded.
9. SK7.SI loaded.
10. CMOU.SI loaded.
11. AJBU.SI loaded.
12. SV3U.SI loaded.
13. AU8U.SI loaded.
14. TS0U.SI loaded.
15. UD1U.SI loaded.
16. O5RU.SI loaded.
17. BTOU.SI loaded.
18. P40U.SI loaded.
19. A17U.SI loaded.
20. J69U.SI loaded.
21. C61U.SI loaded.
22. ME8U.SI loaded.
23. Q1P.SI loaded.
24. BWCU.SI loaded.
25. CRPU.SI loaded.
26. Q5T.SI loaded.
27. K2LU.SI loaded.
28. M44U.SI loaded.
29. BMGU.SI loaded.
30. A68U.SI loaded.
31. C38U.SI loaded.
32. J91U.SI loaded.
33. AW9U.SI loaded.
34. M1GU.SI loaded.
35. J85.SI loaded.
36. K71U.SI loaded.
37. T82U.SI loaded.
38. N2IU.SI loaded.
39. F25U.SI loaded.


# Tesing output
* Print the dictionary's key

In [7]:
print(dict_stk.keys())

dict_keys(['D5IU.SI', 'RW0U.SI', 'ND8U.SI', 'CNNU.SI', 'BUOU.SI', 'C2PU.SI', 'SK6U.SI', 'ACV.SI', 'SK7.SI', 'CMOU.SI', 'AJBU.SI', 'SV3U.SI', 'AU8U.SI', 'TS0U.SI', 'UD1U.SI', 'O5RU.SI', 'BTOU.SI', 'P40U.SI', 'A17U.SI', 'J69U.SI', 'C61U.SI', 'ME8U.SI', 'Q1P.SI', 'BWCU.SI', 'CRPU.SI', 'Q5T.SI', 'K2LU.SI', 'M44U.SI', 'BMGU.SI', 'A68U.SI', 'C38U.SI', 'J91U.SI', 'AW9U.SI', 'M1GU.SI', 'J85.SI', 'K71U.SI', 'T82U.SI', 'N2IU.SI', 'F25U.SI'])


* Print the head of a stock dataframe

In [8]:
dict_stk['F25U.SI'].head(20)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2007-11-06,4.740240,4.780760,4.699730,4.740240,3.028651,1957305
2007-11-07,4.740240,4.740240,4.699730,4.740240,3.028651,2623726
2007-11-09,4.699730,4.740240,4.659210,4.740240,3.028651,407258
2007-11-12,4.659210,4.659210,4.537670,4.537670,2.899225,204863
2007-11-13,4.456640,4.578180,4.456640,4.537670,2.899225,33321
2007-11-14,4.537670,4.699730,4.537670,4.699730,3.002768,40726
2007-11-15,4.537670,4.618700,4.537670,4.618700,2.950997,50599
2007-11-16,4.618700,4.618700,4.497150,4.497150,2.873335,30853
2007-11-19,4.537670,4.537670,4.456640,4.497150,2.873335,754044
